In [26]:
!pip install --user "numpy<1.24,>=1.18"


  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyfume 0.3.4 requires numpy==1.24.4, but you have numpy 1.23.5 which is incompatible.


In [27]:
!C:\Users\leona\anaconda3\python.exe -m pip install --user FuzzyTM>=0.4.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.4 which is incompatible.


In [22]:
!C:\Users\leona\anaconda3\python.exe -m pip install --user torchvision --ignore-installed tbb --no-warn-script-location


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
scipy 1.10.0 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.0.0 which is incompatible.



  Using cached torchvision-0.18.1-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)
  Using cached numpy-2.0.0-cp310-cp310-win_amd64.whl (16.5 MB)
  Using cached pillow-10.3.0-cp310-cp310-win_amd64.whl (2.5 MB)
  Using cached torch-2.3.1-cp310-cp310-win_amd64.whl (159.8 MB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
  Using cached sympy-1.12.1-py3-none-any.whl (5.7 MB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-win_amd64.whl (17 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [12]:
import torch as th
import numpy as np
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F

from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN

ImportError: cannot import name 'BBB_Linear' from 'Bayesian.BBBLinear' (C:\Users\leona\OneDrive\Documents\GitHub\BayesCNNvsFreqCNN\Bayesian\BBBLinear.py)